In [ ]:
import os
import time
import ipywidgets as widgets
from IPython.display import display
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Widget for directory input
directory_input = widgets.Text(
    value='',
    placeholder='Enter directory path',
    description='Directory:',
    disabled=False
)

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='info',
    orientation='horizontal'
)

# Display widgets
display(directory_input)
display(progress_bar)

# Enhanced dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png', 'gif', 'bmp', 'tiff'],
    'PDFs': ['pdf'],
    'Documents': ['doc', 'docx', 'txt', 'rtf', 'odt'],
    'Spreadsheets': ['xls', 'xlsx', 'ods'],
    'Presentations': ['ppt', 'pptx', 'odp'],
    'Datasets': ['csv', 'xlsx', 'json', 'xml', 'sql'],
    'Videos': ['mp4', 'mkv', 'avi', 'mov', 'wmv'],
    'Audio': ['mp3', 'wav', 'aac', 'flac'],
    'Archives': ['zip', 'rar', 'tar', 'gz', '7z'],
    'Scripts': ['py', 'js', 'sh', 'bat'],
    'Executables': ['exe', 'bin', 'msi'],
    'Web': ['html', 'htm', 'css', 'js']
}

# Create a reverse lookup dictionary for file extensions
extension_to_category = {ext: category for category, extensions in categories.items() for ext in extensions}

def create_directories(directory):
    for category in categories:
        os.makedirs(os.path.join(directory, category), exist_ok=True)

# Function to classify a file
def classify_file(directory, filename):
    # Find the file extension
    extension = os.path.splitext(filename)[-1][1:]

    # Get the category from the reverse lookup dictionary
    category = extension_to_category.get(extension)
    if category:
        # Construct the file paths
        source_path = os.path.join(directory, filename)
        dest_path = os.path.join(directory, category, filename)

        # Move the file
        os.rename(source_path, dest_path)
        print(f'Moved {filename} to {category}')

# Classify all existing files in the directory
def classify_existing_files(directory):
    files = os.listdir(directory)
    total_files = len(files)
    for i, filename in enumerate(files):
        classify_file(directory, filename)
        progress_bar.value = int((i + 1) / total_files * 100)

class FileEventHandler(FileSystemEventHandler):
    def __init__(self, directory):
        self.directory = directory
    
    def on_created(self, event):
        if not event.is_directory:
            classify_file(self.directory, os.path.basename(event.src_path))

def start_monitoring(directory):
    # Create necessary directories
    create_directories(directory)
    
    # Classify existing files
    classify_existing_files(directory)

    # Set up the event handler
    event_handler = FileEventHandler(directory)
    observer = Observer()
    observer.schedule(event_handler, directory, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Start the monitoring when directory path is provided
def on_directory_input_change(change):
    directory = change['new']
    if os.path.isdir(directory):
        progress_bar.value = 0
        start_monitoring(directory)

directory_input.observe(on_directory_input_change, names='value')


In [1]:
import os
import time
import json
import datetime
import shutil
import ipywidgets as widgets
from IPython.display import display
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Widget for directory input
directory_input = widgets.Text(
    value='',
    placeholder='Enter directory path',
    description='Directory:',
    disabled=False
)

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='info',
    orientation='horizontal'
)

# Display widgets
display(directory_input)
display(progress_bar)

# Enhanced dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png', 'gif', 'bmp', 'tiff'],
    'PDFs': ['pdf'],
    'Documents': ['doc', 'docx', 'txt', 'rtf', 'odt'],
    'Spreadsheets': ['xls', 'xlsx', 'ods'],
    'Presentations': ['ppt', 'pptx', 'odp'],
    'Datasets': ['csv', 'xlsx', 'json', 'xml', 'sql', 'jsonl', 'ftm.json'],
    'Videos': ['mp4', 'mkv', 'avi', 'mov', 'wmv'],
    'Audio': ['mp3', 'wav', 'aac', 'flac'],
    'Archives': ['zip', 'rar', 'tar', 'gz', '7z'],
    'Scripts': ['py', 'js', 'sh', 'bat'],
    'Executables': ['exe', 'bin', 'msi'],
    'Web': ['html', 'htm', 'css', 'js']
}

# Create a reverse lookup dictionary for file extensions
extension_to_category = {ext: category for category, extensions in categories.items() for ext in extensions}

def create_directories(directory):
    for category in categories:
        os.makedirs(os.path.join(directory, category), exist_ok=True)

# Function to classify a file
def classify_file(directory, filename):
    # Find the file extension
    extension = os.path.splitext(filename)[-1][1:]

    # Get the category from the reverse lookup dictionary
    category = extension_to_category.get(extension)
    if category:
        # Construct the file paths
        source_path = os.path.join(directory, filename)
        dest_path = os.path.join(directory, category, filename)

        # Move the file
        os.rename(source_path, dest_path)
        print(f'Moved {filename} to {category}')

# Classify all existing files in the directory
def classify_existing_files(directory):
    files = os.listdir(directory)
    total_files = len(files)
    for i, filename in enumerate(files):
        classify_file(directory, filename)
        progress_bar.value = int((i + 1) / total_files * 100)

# Function to create the gathered folder and move category folders into it
def gather_folders(directory):
    gathered_folder_name = f'gathered on {datetime.datetime.now().strftime("%Y-%m-%d")}'
    gathered_folder_path = os.path.join(directory, gathered_folder_name)
    os.makedirs(gathered_folder_path, exist_ok=True)

    for category in categories:
        category_path = os.path.join(directory, category)
        if os.path.exists(category_path):
            shutil.move(category_path, gathered_folder_path)
    
    return gathered_folder_path

# Function to create a JSON file with the directory structure
def create_json_structure(directory, json_path):
    structure = {}
    for root, dirs, files in os.walk(directory):
        relative_path = os.path.relpath(root, directory)
        if relative_path == '.':
            relative_path = ''
        structure[relative_path] = {'dirs': dirs, 'files': files}
    
    with open(json_path, 'w') as json_file:
        json.dump(structure, json_file, indent=4)

class FileEventHandler(FileSystemEventHandler):
    def __init__(self, directory):
        self.directory = directory
    
    def on_created(self, event):
        if not event.is_directory:
            classify_file(self.directory, os.path.basename(event.src_path))

def start_monitoring(directory):
    # Create necessary directories
    create_directories(directory)
    
    # Classify existing files
    classify_existing_files(directory)

    # Gather all category folders
    gathered_folder_path = gather_folders(directory)

    # Create a JSON file with the directory structure
    json_path = os.path.join(gathered_folder_path, 'directory_structure.json')
    create_json_structure(gathered_folder_path, json_path)

    # Set up the event handler
    event_handler = FileEventHandler(directory)
    observer = Observer()
    observer.schedule(event_handler, directory, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Start the monitoring when directory path is provided
def on_directory_input_change(change):
    directory = change['new']
    if os.path.isdir(directory):
        progress_bar.value = 0
        start_monitoring(directory)

directory_input.observe(on_directory_input_change, names='value')


Text(value='', description='Directory:', placeholder='Enter directory path')

IntProgress(value=0, bar_style='info', description='Progress:')

Moved (1) Стрічка LinkedIn.mp4 to Videos
Moved 1203002759730514.20kQ4YStUWMAArBl1nQp_height640.png to Images
Moved 2022-state-of-cryptocurrency-investigations-survey.pdf to PDFs
Moved 21413_fcc1aef91ab038b33e7c61d6d5439fb0.docx to Documents
Moved 2609_MARKED_White Trezor acc all transactions.xlsx to Datasets
Moved 31909_21aa533f6ae2578290434c8ea47428b1.doc to Documents
Moved 3BMEXniirhvGm3rfmZo49J1duij8rg3rmJ.json to Datasets
Moved 5278222165348565702_121.jpg to Images
Moved 5280963625793732751_121.jpg to Images
Moved 5280963625793732752_121.jpg to Images
Moved 5330296607998066891_121.jpg to Images
Moved 5330296607998066919_120.jpg to Images
Moved 5_1_SR Amount DebitCreditUsage.xls to Spreadsheets
Moved 7z2201-x64.exe to Executables
Moved ACFrOgBL1diqkT2dZ2k2oWEaZhI2ypdUdXxC0JniQVcdwi0fV6aVPFQNazId7g0QD5KQ-bM0qbHGHMqYRTLqvw_YsdRNKxiq_cAx_05oIrlAuzIdbvlWMgRFw0Oxhs9Dp4JCiZknWKSC6jDsYwrw.pdf to PDFs
Moved aida64extreme675.zip to Archives
Moved Appearance Menu Mod-790-1-14-5-1663839145.zip